Implemented both voltage and threshold equation. 
Issues: Not sure if Nengo's calculations for delta_t or spike time in function step_math are necessary, so I commented them out as the log calculation for spike time caused errors frequently; changed gain to 1 and bias to zero; still haven't figured out how to simulate custom neuron type on loihi.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import math
from scipy.spatial.distance import pdist, squareform
from IPython.display import clear_output
%matplotlib inline

import nengo
from nengo.params import Parameter, NumberParam, NdarrayParam
from nengo.neurons import settled_firingrate

def adjacency(num, ri, ro, nx):
    """Calculate intra-layer adjacency matrix encoding the spacial connectivity 
    of neurons within the layer.

    Parameters
    ----------
    num : int
        Number of neurons in layer
    ri : float
        Excitation radius
    ro : float
        Inhibition radius
    """
    D = squareform(pdist(nx))
    S = 30 * (D < 3) - (10 * (D > 6) *  np.exp(-D/10))
    S = S - np.diag(np.diag(S)) 
    return S    

class CustomLIF(nengo.neurons.NeuronType):
    """Spiking version of the leaky integrate-and-fire (LIF) neuron model.

    Parameters
    ----------
    tau_rc : float
        Membrane RC time constant, in seconds. Affects how quickly the membrane
        voltage decays to zero in the absence of input (larger = slower decay).
    tau_ref : float
        Absolute refractory period, in seconds. This is how long the
        membrane voltage is held at zero after a spike.
    min_voltage : float
        Minimum value for the membrane voltage. If ``-np.inf``, the voltage
        is never clipped.
    amplitude : float
        Scaling factor on the neuron output. Corresponds to the relative
        amplitude of the output spikes of the neuron.
    num : int
        Number of neurons in the layer.
    S : ndarray
        Intra-layer adjacency matrix.
    """

    probeable = ("spikes", "voltage", "refractory_time", "threshold")

    min_voltage = NumberParam("min_voltage", high=0)
    tau_rc = NumberParam("tau_rc", low=0, low_open=True)
    tau_ref = NumberParam("tau_ref", low=0)
    amplitude = NumberParam("amplitude", low=0, low_open=True)
    num = NumberParam("num")
    S = NdarrayParam("S")  # adjacency matrix
    tau_th = NumberParam("tau_th")
    th_plus = NumberParam("th_plus")
    v_th = NumberParam("v_th")
    v_reset = NdarrayParam("v_reset")  # noise on activity field
    nx = NdarrayParam("nx")
    
    def __init__(
        self, 
        S, 
        num, 
        tau_rc, 
        tau_th,
        th_plus,
        v_th,
        v_reset,
        nx,
        tau_ref=0.002, 
        min_voltage=0, 
        amplitude=1,
    ):
        super().__init__()
        self.tau_rc = tau_rc
        self.tau_ref = tau_ref
        self.amplitude = amplitude
        self.min_voltage = min_voltage
        self.num = num
        self.S = S
        self.nx = nx
        self.tau_th=tau_th
        self.th_plus=th_plus
        self.v_th=v_th
        self.v_reset=v_reset

    def gain_bias(self, max_rates, intercepts):
        """Analytically determine gain, bias."""
        gain = np.ones((self.num,))
        bias = np.zeros((self.num,))
        return gain, bias
    

    def max_rates_intercepts(self, gain, bias):
        """Compute the inverse of gain_bias."""
        intercepts = (1 - bias) / gain
        max_rates = 1.0 / (
            self.tau_ref - self.tau_rc * np.log1p(1.0 / (gain * (intercepts - 1) - 1))
        )
        if not np.all(np.isfinite(max_rates)):
            warnings.warn(
                "Non-finite values detected in `max_rates`; this "
                "probably means that `gain` was too small."
            )
        return max_rates, intercepts


    def rates(self, x, gain, bias):
        """Always use LIFRate to determine rates."""
        J = self.current(x, gain, bias)
        out = np.zeros_like(J)
        # Use LIFRate's step_math explicitly to ensure rate approximation
        LIFRate.step_math(self, dt=1, J=J, output=out)
        return out


    def step_math(self, dt, J, spiked, voltage, refractory_time, threshold):        
        # reduce all refractory times by dt
        refractory_time -= dt
        
        # step voltage
        U = np.matmul(spiked, S)
        eta = 3*np.random.rand(self.num,)
        dV = (1/self.tau_rc) * (-1*voltage + U + eta)
        voltage[:] += dV * dt
#         print('final voltage = {}'.format(voltage))
                
        # step threshold voltage (theta)
        dTh = (1/self.tau_th)*(self.v_th-threshold)*(1-spiked)+self.th_plus*spiked
        threshold[:] += dTh * dt
#         print('threshold = {}'.format(threshold))
        
        # determine which neurons spiked (set them to 1/dt, else 0)
        spiked_mask = voltage > threshold
        spiked[:] = spiked_mask * (self.amplitude)
#         print('spiked mask= {}'.format(spiked_mask))
        
        # Visualization of Wave
        plt.scatter(nx[:,0],nx[:,1], color = 'k')
        plt.title('Ret-Wave')
        fired = np.argwhere(spiked)
        plt.scatter(nx[fired,0],nx[fired,1], color = 'r')

        plt.show()
        clear_output(wait=True)
        

        # set spiked voltages to v_reset, refractory times to tau_ref, and
        # rectify negative voltages to a floor of min_voltage
        voltage[voltage < self.min_voltage] = self.min_voltage 
        voltage[spiked_mask] = self.v_reset[spiked_mask]
        refractory_time[spiked_mask] = self.tau_ref

In [2]:
from nengo.builder.operator import Operator

class SimCustomLIF(Operator):
    """Set a neuron model output for the given input current.

    Implements ``neurons.step_math(dt, J, output, *states)``.

    Parameters
    ----------
    neurons : NeuronType
        The `.NeuronType`, which defines a ``step_math`` function.
    J : Signal
        The input current.
    output : Signal
        The neuron output signal that will be set.
    states : list, optional
        A list of additional neuron state signals set by ``step_math``.
    tag : str, optional
        A label associated with the operator, for debugging purposes.

    Attributes
    ----------
    J : Signal
        The input current.
    neurons : NeuronType
        The `.NeuronType`, which defines a ``step_math`` function.
    output : Signal
        The neuron output signal that will be set.
    states : list
        A list of additional neuron state signals set by ``step_math``.
    tag : str or None
        A label associated with the operator, for debugging purposes.

    Notes
    -----
    1. sets ``[output] + states``
    2. incs ``[]``
    3. reads ``[J]``
    4. updates ``[]``
    """
    
    def __init__(self, neurons, J, output, states=None, tag=None):
        super().__init__(tag=tag)
        self.neurons = neurons

        self.sets = [output] + ([] if states is None else states)
        self.incs = []
        self.reads = [J]
        self.updates = []

    @property
    def J(self):
        return self.reads[0]

    @property
    def output(self):
        return self.sets[0]

    @property
    def states(self):
        return self.sets[1:]

    def _descstr(self):
        return "%s, %s, %s" % (self.neurons, self.J, self.output)

    def make_step(self, signals, dt, rng):
        J = signals[self.J]
        output = signals[self.output]
        states = [signals[state] for state in self.states]

        def step_simcustomlif():
            self.neurons.step_math(dt, J, output, *states)

        return step_simcustomlif

In [3]:
from nengo.builder import Builder
from nengo.builder.operator import Copy
from nengo.builder.signal import Signal
from nengo.rc import rc


@Builder.register(CustomLIF)
def build_customlif(model, neuron_type, neurons):
    """Builds a `.LIF` object into a model.

    In addition to adding a `.SimNeurons` operator, this build function sets up
    signals to track the voltage and refractory times for each neuron.

    Parameters
    ----------
    model : Model
        The model to build into.
    neuron_type : CustomLIF
        Neuron type to build.
    neuron : Neurons
        The neuron population object corresponding to the neuron type.

    Notes
    -----
    Does not modify ``model.params[]`` and can therefore be called
    more than once with the same `.LIF` instance.
    """

    model.sig[neurons]["voltage"] = Signal(
        shape=neurons.size_in, name="%s.voltage" % neurons
    )
    model.sig[neurons]["refractory_time"] = Signal(
        shape=neurons.size_in, name="%s.refractory_time" % neurons
    )
    model.sig[neurons]["threshold"] = Signal(
        shape=neurons.size_in, name= "%s.threshold" % neurons
    )
    model.add_op(
        SimCustomLIF(
            neurons=neuron_type,
            J=model.sig[neurons]["in"],
            output=model.sig[neurons]["out"],
            states=[
                model.sig[neurons]["voltage"],
                model.sig[neurons]["refractory_time"],
                model.sig[neurons]["threshold"],
            ],
        )
    )

In [4]:
from nengo.utils.matplotlib import rasterplot
# import nengo_loihi

num = 1632

nx = 28*np.random.rand(num,2)
S = adjacency(num, 3, 6, nx)
rand = np.random.randn(num,)
v_reset = 0+0.1*rand*rand

# nengo_loihi.set_defaults()


model = nengo.Network(label='2D Representation', seed=10)
process = nengo.processes.WhiteNoise(
    dist=nengo.dists.Gaussian(0, .01), seed=1)
with model:
    a = nengo.Ensemble(num, dimensions=2, neuron_type=CustomLIF(S, num, 2, 60, 9, 1, v_reset, nx))
    b = nengo.Ensemble(num, dimensions=2, neuron_type=CustomLIF(S, num, 2, 60, 9, 1, v_reset, nx))
#     conn = nengo.Connection(a, b, learning_rule_type=nengo.PES(learning_rate=2e-4))
    spikes_probe = nengo.Probe(a.neurons, 'spikes')
    voltage_probe = nengo.Probe(a.neurons, 'voltage')
    threshold_probe = nengo.Probe(a.neurons, 'threshold')
    b_spikes = nengo.Probe(b.neurons, 'spikes')
    b_voltage = nengo.Probe(b.neurons, 'voltage')
with nengo.Simulator(model) as sim:
    sim.run(.2)
    
# Custom LIF neurons
# plt.figure(figsize=(12, 6))
# plt.plot(sim.trange(), sim.data[voltage_probe])
# plt.xlabel('time [s]')
# plt.ylabel('voltage')

# plt.figure(figsize=(12, 6))
# plt.plot(sim.trange(), sim.data[threshold_probe])
# plt.xlabel('time [s]')
# plt.ylabel('threshold voltage')

# plt.figure(figsize=(12, 6))
# rasterplot(sim.trange(), sim.data[spikes_probe])
# plt.xlabel('time [s]')
# plt.ylabel('Neuron number')